In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers bitsandbytes sentencepiece accelerate guidance --upgrade -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 57.6 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.5 MB/s eta 0:00:00


In [8]:
!wget -nc https://people.eecs.berkeley.edu/~hendrycks/data.tar

--2024-06-04 10:30:43--  https://people.eecs.berkeley.edu/~hendrycks/data.tar
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166184960 (158M) [application/x-tar]
Saving to: ‘data.tar’

data.tar            100%[===================>] 158.49M  20.4MB/s    in 69s     

2024-06-04 10:31:53 (2.30 MB/s) - ‘data.tar’ saved [166184960/166184960]



In [ ]:
import tarfile
unzip_path = '.'
tar = tarfile.open('data.tar')
tar.extractall(path=unzip_path)
tar.close()

In [10]:
!pip install --upgrade transformers -qq
!pip install accelerate
!pip install groq
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00


In [11]:
import torch
import numpy as np
from sklearn.isotonic import IsotonicRegression
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline
from transformers import BitsAndBytesConfig
import torch.nn.functional as F
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown
import google.generativeai as genai
from bert_score import score
import os
from groq import Groq
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import json
import re
import concurrent.futures
from tqdm import tqdm
from yaml import safe_load

In [12]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
# Define models and tokenizers
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)
tokenizer_mistral.pad_token = tokenizer_mistral.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

mistral_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    device_map="auto",
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model_name2 = 'stabilityai/StableBeluga-13B'
tokenizer_beluga = AutoTokenizer.from_pretrained(model_name2)
tokenizer_beluga.pad_token = tokenizer_beluga.eos_token

beluga_model = AutoModelForCausalLM.from_pretrained(
    model_name2,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    low_cpu_mem_usage=True,
    device_map="auto",
)

In [14]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(filtered_tokens)

In [15]:
# Utility functions
def load_questions(file_path):
    questions = []
    with open(file_path, 'r') as f:
        for line in f:
            questions.append(json.loads(line.strip()))
    return questions

def load_model_answers(dir_path):
    model_answers = {}
    for file_name in os.listdir(dir_path):
        model_name = file_name.split('.')[0]
        model_answers[model_name] = {}
        with open(os.path.join(dir_path, file_name), 'r') as f:
            for line in f:
                answer = json.loads(line.strip())
                question_id = answer['question_id']
                model_answers[model_name][question_id] = answer
    return model_answers

def get_endpoint(endpoint_config):
    # Implementation depends on the format of your API config file
    return endpoint_config

def make_config(file_path):
    with open(file_path, 'r') as f:
        return safe_load(f)

def chat_completion_groq(model, conv, temperature, max_tokens, api_key):
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=conv,
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

def get_score(judgment, pattern, pairwise=True):
    matches = pattern.findall(judgment)
    matches = [m for m in matches if m != ""]
    if len(set(matches)) == 0:
        return None, True
    elif len(set(matches)) == 1:
        if pairwise:
            return matches[0].strip("\n"), False
        return int(matches[0])
    else:
        return None, False

In [16]:
client = Groq(api_key="YOUR_API")

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the meaning of life? How modern Ai is affeting the real life and what will be the Ai outcomes in future?",
        }

    ],
    model="llama3-70b-8192",
)

print(chat_completion.choices[0].message.content)

**What is the meaning of life?**

This is a question that has puzzled philosophers, theologians, and scientists for centuries. While there is no one definitive answer, here are a few perspectives:

1. **Biological perspective**: From a biological standpoint, the meaning of life could be simply to survive, reproduce, and pass on one's genes to the next generation.
2. **Philosophical perspective**: Philosophers have proposed various answers, such as:
	* **Hedonism**: The pursuit of pleasure and happiness.
	* **Eudaimonia**: Living a virtuous and fulfilling life.
	* **Existentialism**: Finding one's own purpose and meaning in life.
3. **Religious perspective**: Many religions provide answers to the meaning of life, such as:
	* **Christianity**: The purpose of life is to love and serve God, and to attain salvation.
	* **Buddhism**: The goal is to achieve enlightenment and end suffering.
4. **Psychological perspective**: According to psychologists, the meaning of life can be found in:
	* **

## Class Pipeline

In [ ]:
class EvaluationPipeline:
    def __init__(self, api_key, judge_model, regex_pattern, temperature=0.8, max_tokens=300, pairwise=True):
        self.api_key = api_key
        self.judge_model = judge_model
        self.regex_pattern = re.compile(regex_pattern)
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.pairwise = pairwise
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

    def get_mistral_answer(self, prompt, include_stopwords=True):
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8,
            do_sample=True, output_scores=True, return_dict_in_generate=True
        )

        response_ids = outputs.sequences[0]
        response_text = tokenizer_mistral.decode(response_ids, skip_special_tokens=True)

        logits = outputs.scores  # Logits of the generated tokens
        softmax_probs = torch.softmax(torch.stack(logits), dim=-1)

        tokens = tokenizer_mistral.convert_ids_to_tokens(response_ids)
        if include_stopwords:
            token_confidences = [prob.max().item() for prob in softmax_probs]
        else:
            token_confidences = [prob.max().item() for prob, token in zip(softmax_probs, tokens) if token.lower() not in stop_words]

        mean_confidence = np.mean(token_confidences)

        return response_text, mean_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model="llama3-70b-8192",
        )

        llama3_response = chat_completion.choices[0].message.content.strip()
        return llama3_response

    def parse_evaluation(self, evaluation):
        if "1 point" in evaluation:
            return 1
        return 0

    def generate_reference_text(self, prompt):
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8, do_sample=True
        )

        reference_text = tokenizer_mistral.decode(outputs[0], skip_special_tokens=True)
        return reference_text

    def calculate_bertscore(self, references, candidates):
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()

    def evaluate(self, prompts):
        self.prompts = prompts  # Ensure prompts are stored
        for prompt in prompts:
            response, confidence = self.get_mistral_answer(prompt)
            self.responses.append(response)
            self.confidences.append(confidence)
            reference_text = self.generate_reference_text(prompt)
            self.references.append(reference_text)
            judgement = self.judge_answer(response, prompt)
            accuracy = self.parse_evaluation(judgement)
            self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

    def calculate_ece(self):
        # Ensure all lists have the same length
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        # Normalize confidence scores
        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        # Bin the normalized confidence scores
        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        # Calculate accuracy for each bin
        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        # Drop bins with NaN values
        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        # Ensure lengths match
        bin_confidences = bin_confidences[:len(bin_accuracies)]

        # Compute ECE
        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

    def plot_reliability_diagram(self, bins, bin_accuracies):
        plt.figure(figsize=(10, 6))
        plt.plot((bins[:-1] + bins[1:]) / 2, bin_accuracies, marker='o', label='Accuracy per bin')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect calibration')
        plt.xlabel('Confidence')
        plt.ylabel('Accuracy')
        plt.title('Reliability Diagram')
        plt.legend()
        plt.grid(True)
        plt.show()

In [ ]:
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model="llama3-70b-8192", regex_pattern=r"(correct|incorrect)")
prompts = ["What is nuclear fusion?", "Tell me about the current AI situation in the world?", "How does a combustion engine work?"]
pipeline.evaluate(prompts)

In [ ]:
data, bins, bin_accuracies = pipeline.calculate_ece()

### With or Without Stopwords

In [ ]:
few_shot_examples = """
Q: What is nuclear fusion?
A: Nuclear fusion is a reaction in which two atomic nuclei combine to form a heavier nucleus, releasing energy in the process.

Q: How does a combustion engine work?
A: A combustion engine works by burning fuel in a combustion chamber to produce mechanical energy that drives the engine.
"""

def get_mistral_answer_few_shot(prompt):
    few_shot_prompt = few_shot_examples + f"\nQ: {prompt}\nA:"
    response, confidence = get_mistral_answer(few_shot_prompt)
    return response, confidence

# Example usage:
few_shot_response, few_shot_confidence = get_mistral_answer_few_shot("Tell me about the current AI situation in the world.")
print(f"Few-shot response: {few_shot_response}")
print(f"Few-shot confidence: {few_shot_confidence}")

### Class pipeline for mmlu dataset

In [20]:
import re

class EvaluationPipeline:
    def __init__(self, api_key, judge_model, regex_pattern, temperature=0.8, max_tokens=300, pairwise=True):
        self.api_key = api_key
        self.judge_model = judge_model
        self.regex_pattern = re.compile(regex_pattern)
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.pairwise = pairwise

    def get_mistral_answer(self, prompt, include_stopwords=True):
        tokenizer_mistral = AutoTokenizer.from_pretrained(model_name)
        tokenizer_mistral.pad_token = tokenizer_mistral.eos_token
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8,
            do_sample=True, output_scores=True, return_dict_in_generate=True
        )

        response_ids = outputs.sequences[0]
        response_text = tokenizer_mistral.decode(response_ids, skip_special_tokens=True)

        logits = outputs.scores  # Logits of the generated tokens
        softmax_probs = torch.softmax(torch.stack(logits), dim=-1)

        tokens = tokenizer_mistral.convert_ids_to_tokens(response_ids)
        if include_stopwords:
            token_confidences = [prob.max().item() for prob in softmax_probs]
        else:
            token_confidences = [prob.max().item() for prob, token in zip(softmax_probs, tokens) if token.lower() not in stop_words]

        mean_confidence = np.mean(token_confidences)

        return response_text, mean_confidence

    def judge_answer(self, response, prompt):
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": f"""
                        Review the user’s question and the corresponding response using the binary scoring system described below.
                        - 0 points: The response is incorrect or does not address the user’s question.
                        - 1 point: The response is correct and addresses the user’s question.

                        User: {prompt}
                        Response: {response}
                        """
                }
            ],
            model="llama3-70b-8192",
        )

        llama3_response = chat_completion.choices[0].message.content.strip()
        return llama3_response

    def parse_evaluation(self, evaluation):
        if "1 point" in evaluation:
            return 1
        return 0

    def generate_reference_text(self, prompt):
        inputs = tokenizer_mistral.encode_plus(
            prompt, return_tensors='pt', max_length=1024, truncation=True, padding='max_length'
        ).to(mistral_model.device)

        outputs = mistral_model.generate(
            inputs['input_ids'], attention_mask=inputs['attention_mask'],
            max_new_tokens=300, num_return_sequences=1, temperature=0.8, do_sample=True
        )

        reference_text = tokenizer_mistral.decode(outputs[0], skip_special_tokens=True)
        return reference_text

    def calculate_bertscore(self, references, candidates):
        P, R, F1 = score(candidates, references, lang="en", model_type="bert-base-uncased")
        return P.mean().item(), R.mean().item(), F1.mean().item()


    def evaluate_from_csv(self, csv_file_path):
        # Clear data before processing each file
        self.prompts = []
        self.responses = []
        self.references = []
        self.confidences = []
        self.accuracies = []

        df = pd.read_csv(csv_file_path)
        self.prompts = df.iloc[:, 0].tolist()  # Take only the first column (questions)

        for prompt in self.prompts:
            response, confidence = self.get_mistral_answer(prompt)
            self.responses.append(response)
            self.confidences.append(confidence)
            reference_text = self.generate_reference_text(prompt)
            self.references.append(reference_text)
            judgement = self.judge_answer(response, prompt)
            accuracy = self.parse_evaluation(judgement)
            self.accuracies.append(accuracy)

        precision, recall, f1 = self.calculate_bertscore(self.references, self.responses)
        print(f"BERTScore - Precision: {precision}, Recall: {recall}, F1: {f1}")

        data, bins, bin_accuracies = self.calculate_ece()
        data.to_csv("results.csv", index=False)  # Save results to CSV
        return data, bins, bin_accuracies

    def evaluate_folder(self, folder_path):
        results = {}  # Store ECE values for each class
        for filename in os.listdir(folder_path):
            if filename.endswith(".csv"):
                filepath = os.path.join(folder_path, filename)
                class_name = filename[:-4]  # Extract class name from filename
                print(f"Evaluating {class_name}...")
                data, bins, bin_accuracies = self.evaluate_from_csv(filepath)

                # Align shapes and calculate ECE
                bin_confidences = (bins[:-1] + bins[1:]) / 2  # Confidences for each bin
                valid_bins = bin_accuracies.dropna().index  # Bins with data
                bin_accuracies = bin_accuracies[valid_bins]
                bin_proportions = data['bin'].value_counts(normalize=True)[valid_bins] # Get proportions for valid bins
                bin_confidences = bin_confidences[valid_bins] # Select confidences for valid bins

                ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)

                results[class_name] = ece
                print(f"{class_name} ECE: {ece}")

        # Save results to CSV
        results_df = pd.DataFrame({'Class': results.keys(), 'ECE': results.values()})
        results_df.to_csv("results.csv", index=False)


    def calculate_ece(self):
        # Ensure all lists have the same length
        if len(self.prompts) == len(self.responses) == len(self.confidences) == len(self.accuracies):
            data = pd.DataFrame({
                'prompt': self.prompts,
                'response': self.responses,
                'confidence': self.confidences,
                'rating': self.accuracies
            })
        else:
            raise ValueError("All arrays must be of the same length")

        # Normalize confidence scores
        data['confidence_normalized'] = data['confidence'] / data['confidence'].max()

        # Bin the normalized confidence scores
        bins = np.linspace(0, 1, 11)
        data['bin'] = pd.cut(data['confidence_normalized'], bins=bins, labels=False, include_lowest=True)

        # Calculate accuracy for each bin
        bin_accuracies = data.groupby('bin')['rating'].mean()
        bin_proportions = data['bin'].value_counts(normalize=True)

        # Drop bins with NaN values
        valid_bins = bin_accuracies.dropna().index
        bin_accuracies = bin_accuracies[valid_bins]
        bin_proportions = bin_proportions[valid_bins]
        bin_confidences = (bins[:-1] + bins[1:]) / 2
        bin_confidences = bin_confidences[valid_bins]

        # Ensure lengths match
        bin_confidences = bin_confidences[:len(bin_accuracies)]

        # Compute ECE
        ece = np.sum(np.abs(bin_accuracies - bin_confidences) * bin_proportions)
        print(f"Expected Calibration Error (ECE): {ece}")

        return data, bins, bin_accuracies

#     def plot_reliability_diagram(self, bins, bin_accuracies):
#         plt.figure(figsize=(10, 6))
#         plt.plot((bins[:-1] + bins[1:]) / 2, bin_accuracies, marker='o', label='Accuracy per bin')
#         plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect calibration')
#         plt.xlabel('Confidence')
#         plt.ylabel('Accuracy')
#         plt.title('Reliability Diagram')
#         plt.legend()
#         plt.grid(True)
#         plt.show()

In [21]:
pipeline = EvaluationPipeline(api_key="Your_api_key", judge_model="llama3-70b-8192", regex_pattern=r"(correct|incorrect)")
pipeline.evaluate_folder("/content/drive/MyDrive/data/dev")

Evaluating medical_genetics_dev...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BERTScore - Precision: 0.7204489707946777, Recall: 0.7340354323387146, F1: 0.7271149754524231
Expected Calibration Error (ECE): 0.050000000000000044
medical_genetics_dev ECE: 0.050000000000000044
Evaluating anatomy_dev...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BERTScore - Precision: 0.6905605792999268, Recall: 0.6710290312767029, F1: 0.6803138256072998
Expected Calibration Error (ECE): 0.29999999999999993
anatomy_dev ECE: 0.29999999999999993
Evaluating philosophy_dev...


...

BERTScore - Precision: 0.6215066313743591, Recall: 0.64439457654953, F1: 0.6326566338539124
Expected Calibration Error (ECE): 0.09999999999999998
virology_dev ECE: 0.09999999999999998
Evaluating world_religions_dev...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BERTScore - Precision: 0.7294026017189026, Recall: 0.729739785194397, F1: 0.7289401292800903
Expected Calibration Error (ECE): 0.050000000000000044
world_religions_dev ECE: 0.050000000000000044
Evaluating moral_scenarios_dev...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


BERTScore - Precision: 0.8285967111587524, Recall: 0.8351159691810608, F1: 0.8308065533638
Expected Calibration Error (ECE): 0.44999999999999996
moral_scenarios_dev ECE: 0.44999999999999996
